# Media Library

> Unified facade for media file discovery, mounting, and access

In [ ]:
#| default_exp media.library

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Optional
from fastcore.basics import patch
from fasthtml.common import APIRouter 

from cjm_fasthtml_workflow_transcription_single_file.media.config import MediaConfig
from cjm_fasthtml_workflow_transcription_single_file.media.models import MediaFile
from cjm_fasthtml_workflow_transcription_single_file.media.scanner import MediaScanner
from cjm_fasthtml_workflow_transcription_single_file.media.mounter import MediaMounter

## MediaLibrary Class

A unified facade that combines `MediaScanner` and `MediaMounter` functionality. Provides a simple interface for browsing media files and accessing their URLs.

In [ ]:
#| export
class MediaLibrary:
    """Unified interface for media scanning, mounting, and browsing.

    This class provides a facade over the individual media components,
    offering a simple API for workflow integration.

    Example usage:
        config = MediaConfig(directories=["/path/to/media"])
        library = MediaLibrary(config)

        # Mount directories to app
        library.mount(app)

        # Scan for files
        files = library.scan()

        # Get URL for a file
        url = library.get_url(files[0].path)

        # Create pagination for browsing
        pagination = library.create_pagination("my_pagination", "content-id")
    """

    def __init__(self, config: MediaConfig):
        """Initialize the media library.

        Args:
            config: Media configuration with directories and settings.
        """
        self.config = config
        self.scanner = MediaScanner(config)
        self.mounter = MediaMounter()
        self._pagination = None
        self._pagination_router = None

    @property
    def pagination(self):
        """Access to the pagination instance."""
        return self._pagination

    @property
    def pagination_router(self) -> Optional[APIRouter]:
        """Access to the pagination router."""
        return self._pagination_router

    @property
    def file_selection_pagination(self):
        """Access to the file selection pagination instance."""
        return getattr(self, '_file_selection_pagination', None)

    @property
    def file_selection_router(self) -> Optional[APIRouter]:
        """Access to the file selection pagination router."""
        return getattr(self, '_file_selection_router', None)

In [ ]:
#| export
@patch
def mount(self: MediaLibrary, app) -> None:
    """Mount media directories to app for static file serving.

    This must be called after the FastHTML app is created.

    Args:
        app: FastHTML/Starlette application instance.
    """
    self.mounter.mount(app, self.config.directories)

In [ ]:
#| export
@patch
def scan(self: MediaLibrary, force_refresh: bool = False) -> List[MediaFile]:
    """Scan for media files.

    Args:
        force_refresh: Force a fresh scan, ignoring cache.

    Returns:
        List of MediaFile objects.
    """
    return self.scanner.scan(force_refresh)

In [ ]:
#| export
@patch
def get_transcribable_files(self: MediaLibrary) -> List[MediaFile]:
    """Get files suitable for transcription (audio and video only).

    Returns:
        List of MediaFile objects with media_type 'audio' or 'video'.
    """
    files = self.scan()
    return [f for f in files if f.media_type in ["audio", "video"]]

In [ ]:
#| export
@patch
def get_url(self: MediaLibrary, file_path: str) -> Optional[str]:
    """Get URL for a media file.

    Args:
        file_path: Full path to the media file.

    Returns:
        URL path to access the file, or None if not in a mounted directory.
    """
    return self.mounter.get_url(file_path)

In [ ]:
#| export
@patch
def clear_cache(self: MediaLibrary) -> None:
    """Clear the scan cache."""
    self.scanner.clear_cache()

In [ ]:
#| export
@patch
def get_summary(self: MediaLibrary) -> dict:
    """Get summary statistics for scanned files.

    Returns:
        Dictionary with file counts, sizes, and breakdowns.
    """
    return self.scanner.get_summary()

In [ ]:
#| export
@patch
def create_pagination(
    self: MediaLibrary,
    pagination_id: str,
    content_id: str,
    preview_route_func=None,
    modal_id: str = "sf-media-preview"
):
    """Create a pagination instance for browsing media files.

    Args:
        pagination_id: Unique identifier for this pagination instance.
        content_id: HTML ID for the content area.
        preview_route_func: Optional function to generate preview route URL.
        modal_id: ID for the preview modal.

    Returns:
        Configured Pagination instance.
    """
    self._pagination = create_media_pagination(
        pagination_id=pagination_id,
        scanner=self.scanner,
        mounter=self.mounter,
        items_per_page=self.config.items_per_page,
        content_id=content_id,
        preview_route_func=preview_route_func,
        modal_id=modal_id
    )
    return self._pagination

In [ ]:
#| export
@patch
def get_pagination_router(self: MediaLibrary, prefix: str) -> Optional[APIRouter]:
    """Get the pagination router for registration with the app.

    Must be called after create_pagination().

    Args:
        prefix: URL prefix for pagination routes.

    Returns:
        APIRouter for pagination, or None if pagination not created.
    """
    if self._pagination:
        self._pagination_router = self._pagination.create_router(prefix=prefix)
        return self._pagination_router
    return None

In [ ]:
#| export
@patch
def create_file_selection_pagination(
    self: MediaLibrary,
    pagination_id: str,
    content_id: str,
    preview_url_func=None,
    preview_target_id: str = None
):
    """Create a pagination instance for file selection table.

    This pagination is specifically for the file selection step,
    rendering a table with radio buttons for selection.

    Args:
        pagination_id: Unique identifier for this pagination instance.
        content_id: HTML ID for the content area.
        preview_url_func: Function that takes file index and returns preview URL.
        preview_target_id: HTML ID to target for preview modal.

    Returns:
        Configured Pagination instance for file selection.
    """
    self._file_selection_pagination = create_file_selection_pagination(
        pagination_id=pagination_id,
        scanner=self.scanner,
        items_per_page=self.config.items_per_page,
        content_id=content_id,
        preview_url_func=preview_url_func,
        preview_target_id=preview_target_id
    )
    return self._file_selection_pagination

In [ ]:
#| export
@patch
def get_file_selection_router(self: MediaLibrary, prefix: str) -> Optional[APIRouter]:
    """Get the file selection pagination router.

    Must be called after create_file_selection_pagination().

    Args:
        prefix: URL prefix for pagination routes.

    Returns:
        APIRouter for file selection pagination, or None if not created.
    """
    if self._file_selection_pagination:
        self._file_selection_router = self._file_selection_pagination.create_router(prefix=prefix)
        return self._file_selection_router
    return None

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()